[한국전력공사\_공동주택 전력·기상 융합데이터\_20221231 | 공공데이터포털](https://www.data.go.kr/data/15120804/fileData.do)

한국전력공사_공동주택 전력·기상 융합데이터
한국전력공사와 기상청이 공동주택 부하관리관련 연구개발 활성화를 목적으로 제공하는 데이터셋입니다.
데이터셋은 기상청 동네예보 격자단위 시간대별 공동주택 전력부하와 기상관측값을 포함하고 있습니다.
공동주택 전력부하는 각 연도별 결측치가 없는 단지에 한하여 합계한 통계값이며, 격자내 공동주택이 10개 이상인 격자에 대해서만 제공하고 있습니다.

[2024 날씨 빅데이터콘테스트 설명동영상(과제4, 전력분야) - YouTube](https://www.youtube.com/watch?v=9xCBKNrymig&t=265s)


---

**| 학습 데이터(전력)**

-   **제공기간** : 2020년, 2021년, 2022년 (총 3개년)
-   **자료설명** : 원격검침이 이루어지는 공동주택 중 각 연도별 결측치가 없는 단지를 대상으로 기상예보 격자별로 산출된 전력 통계값. 격자내 공동주택이 10개 이상인 격자에 대해서만 공개함.

**컬럼 정보**

| 변수명        | 변수 설명      | 항목설명                                                                                                |
| :------------ | :------------- | :------------------------------------------------------------------------------------------------------ |
| TM            | 날짜           | 공동주택 전력부하 측정 날짜(시간포함, 단위(0~23시))                                                          |
| HH24          | 시간           | 공동주택 전력부하 측정 시간(1~24), 5시는 4시 1분~5시 00분까지의 전력부하를 의미                                  |
| weekday       | 요일           | 요일을 숫자형식으로 표시 월요일(0)~일요일(6)                                                                 |
| week\_name    | 주중 주말      | 주중 주말을 숫자형식으로 표시 주중(0), 주말(1)                                                               |
| SUM\_QCTR     | 계약전력합계   | 해당격자의 전력통계 산출에 포함된 공동주택의 계약전력 합계                                                       |
| N             | 공동주택 수    | 해당격자의 전력통계 산출에 포함된 공동주택의 수, 단위(단지)                                                      |
| SUM\_LOAD     | 전력수요 합계  | 해당격자/시각에 측정된 공동주택의 전력수요 합계                                                                |
| N\_MEAN\_LOAD | 전력부하량 평균 | 격자내 총 전력부하량을 아파트 수로 나누어 격자의 평균 부하량을 산출                                                 |
| elec          | 전력기상지수   | 해당격자의 공동주택의 연평균 부하량을 100으로 했을 때, 해당 시각에 예상되는 부하량을 상대적인 수치로 표현                     |

---

## 전력부하 합계 예측 모델 

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import datetime

In [ ]:
data_folder = 'data'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

df_year = []
for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    df_temp = pd.read_csv(file_path, encoding="cp949")
    df_year.append(df_temp)

df = pd.concat(df_year)
df.shape

In [ ]:
df

In [ ]:
df = df.sort_values(by=['연도', '월', '일', '시']).reset_index(drop=True)
df.shape

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# 연도별 전력부하합계 변화
plt.figure(figsize=(8, 5))
df.groupby('연도')['전력부하합계'].mean().plot(marker='o')
plt.title('연도별 평균 전력부하합계')
plt.ylabel('평균 전력부하합계')
plt.xlabel('연도')
plt.grid(True)


In [ ]:
plt.figure(figsize=(8,5))
df.groupby('월')['전력부하합계'].mean().plot(marker='o', title='월별 평균 전력부하합계')
plt.show()


In [ ]:
# 시간대별 전력부하합계 변화
plt.figure(figsize=(8, 5))
df.groupby('시')['전력부하합계'].mean().plot(marker='o')
plt.title('시간대별 평균 전력부하합계')
plt.ylabel('평균 전력부하합계')
plt.xlabel('시')
plt.grid(True)
plt.show()


In [ ]:

# 기온, 습도, 풍속과 전력부하합계의 관계 산점도
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(x='기온', y='전력부하합계', data=df.sample(10000), ax=axes[0], alpha=0.3)
axes[0].set_title('기온 vs 전력부하합계')
sns.scatterplot(x='상대습도', y='전력부하합계', data=df.sample(10000), ax=axes[1], alpha=0.3)
axes[1].set_title('상대습도 vs 전력부하합계')
sns.scatterplot(x='풍속', y='전력부하합계', data=df.sample(10000), ax=axes[2], alpha=0.3)
axes[2].set_title('풍속 vs 전력부하합계')
plt.tight_layout()
plt.show()


In [ ]:
corr = df.corr()
corr

In [ ]:
sns.heatmap(corr, cmap='seismic', mask=np.triu(np.ones_like(corr)))

In [ ]:
df.isnull().sum()

In [19]:
# 결측치 처리 (예시: 평균값 대체)
df['상대습도'] = df['상대습도'].fillna(df['상대습도'].mean())
df['풍속'] = df['풍속'].fillna(df['풍속'].mean())

# 특성 공학
# 시간 순환 특성
df['시간_sin'] = np.sin(2 * np.pi * df['시'] / 24)
df['시간_cos'] = np.cos(2 * np.pi * df['시'] / 24)
df['월_sin'] = np.sin(2 * np.pi * df['월'] / 12)
df['월_cos'] = np.cos(2 * np.pi * df['월'] / 12)

# 주말 여부
df['날짜'] = df.apply(lambda x: datetime.datetime(int(x['연도']), int(x['월']), int(x['일'])), axis=1)
df['요일'] = df['날짜'].dt.dayofweek
df['주말'] = df['요일'].apply(lambda x: 1 if x >= 5 else 0)

# 계절 더미 변수
df['여름'] = df['월'].apply(lambda x: 1 if x in [6, 7, 8] else 0)
df['겨울'] = df['월'].apply(lambda x: 1 if x in [12, 1, 2] else 0)

# 이용률, 전력밀도
df['이용률'] = df['전력부하합계'] / df['계약전력합계']
df['전력밀도'] = df['전력부하합계'] / df['공동주택수']

# 냉방/난방도일, 쾌적지수 (예시)
df['냉방도일'] = df['기온'].apply(lambda x: max(0, x - 24))
df['난방도일'] = df['기온'].apply(lambda x: max(0, 18 - x))
df['온도쾌적지수'] = df['기온'] * df['상대습도'] / 100

In [20]:
def advanced_feature_engineering(df):
    """
    고급 특성 공학 수행 - 47개 새로운 특성 생성
    """
    print("고급 특성 공학 수행 중...")
    df_features = df.copy()
    
    # 1. 결측치 처리
    df_features['상대습도'] = df_features['상대습도'].fillna(df_features['상대습도'].median())
    df_features['풍속'] = df_features['풍속'].fillna(df_features['풍속'].median())
    df_features['기온'] = df_features['기온'].fillna(df_features['기온'].median())
    
    # 2. 시간 기반 특성 (순환 인코딩)
    df_features['시간_sin'] = np.sin(2 * np.pi * df_features['시'] / 24)
    df_features['시간_cos'] = np.cos(2 * np.pi * df_features['시'] / 24)
    df_features['월_sin'] = np.sin(2 * np.pi * df_features['월'] / 12)
    df_features['월_cos'] = np.cos(2 * np.pi * df_features['월'] / 12)
    df_features['요일_sin'] = np.sin(2 * np.pi * df_features['요일'] / 7)
    df_features['요일_cos'] = np.cos(2 * np.pi * df_features['요일'] / 7)
    
    # 3. 시간대 구분
    df_features['주말'] = (df_features['요일'] >= 5).astype(int)
    df_features['새벽'] = ((df_features['시'] >= 0) & (df_features['시'] < 6)).astype(int)
    df_features['오전'] = ((df_features['시'] >= 6) & (df_features['시'] < 12)).astype(int)
    df_features['오후'] = ((df_features['시'] >= 12) & (df_features['시'] < 18)).astype(int)
    df_features['저녁'] = ((df_features['시'] >= 18) & (df_features['시'] < 24)).astype(int)
    df_features['오전피크'] = ((df_features['시'] >= 8) & (df_features['시'] <= 10)).astype(int)
    df_features['저녁피크'] = ((df_features['시'] >= 18) & (df_features['시'] <= 20)).astype(int)
    
    # 4. 계절 기반 특성
    df_features['봄'] = df_features['월'].isin([3, 4, 5]).astype(int)
    df_features['여름'] = df_features['월'].isin([6, 7, 8]).astype(int)
    df_features['가을'] = df_features['월'].isin([9, 10, 11]).astype(int)
    df_features['겨울'] = df_features['월'].isin([12, 1, 2]).astype(int)
    
    # 5. 기상 기반 특성
    df_features['냉방도일'] = np.maximum(0, df_features['기온'] - 24)
    df_features['난방도일'] = np.maximum(0, 18 - df_features['기온'])
    df_features['불쾌지수'] = 1.8 * df_features['기온'] - 0.55 * (1 - df_features['상대습도']/100) * (1.8 * df_features['기온'] - 26) + 32
    df_features['체감온도'] = df_features['기온'] - 0.4 * (df_features['기온'] - 10) * (1 - df_features['상대습도']/100)
    
    # 6. 극한 기상 조건
    df_features['고온'] = (df_features['기온'] > df_features['기온'].quantile(0.9)).astype(int)
    df_features['저온'] = (df_features['기온'] < df_features['기온'].quantile(0.1)).astype(int)
    df_features['고습도'] = (df_features['상대습도'] > df_features['상대습도'].quantile(0.9)).astype(int)
    df_features['강풍'] = (df_features['풍속'] > df_features['풍속'].quantile(0.9)).astype(int)
    
    # 7. 전력 관련 파생 특성
    df_features['이용률'] = df_features['전력부하합계'] / (df_features['계약전력합계'] + 1e-6)
    df_features['전력밀도'] = df_features['전력부하합계'] / (df_features['공동주택수'] + 1e-6)
    df_features['단지당계약전력'] = df_features['계약전력합계'] / (df_features['공동주택수'] + 1e-6)
    df_features['정규화부하'] = df_features['전력부하합계'] / (df_features['계약전력합계'] * df_features['공동주택수'] / 100 + 1e-6)
    
    # 8. 시계열 지연 특성 (데이터 시간순 정렬 필요)
    df_features = df_features.sort_values('datetime').reset_index(drop=True)
    for lag in [1, 24, 168]:  # 1시간, 1일, 1주일 전
        if lag < len(df_features):
            df_features[f'전력부하_lag{lag}'] = df_features['전력부하합계'].shift(lag)
            df_features[f'기온_lag{lag}'] = df_features['기온'].shift(lag)
    
    # 9. 이동 평균 및 표준편차
    for window in [24, 168]:  # 24시간, 1주일
        if window < len(df_features):
            df_features[f'전력부하_ma{window}'] = df_features['전력부하합계'].rolling(window=window, min_periods=1).mean()
            df_features[f'기온_ma{window}'] = df_features['기온'].rolling(window=window, min_periods=1).mean()
            df_features[f'전력부하_std{window}'] = df_features['전력부하합계'].rolling(window=window, min_periods=1).std()
    
    # 10. 상호작용 특성
    df_features['기온_시간'] = df_features['기온'] * df_features['시']
    df_features['기온_제곱'] = df_features['기온'] ** 2
    df_features['기온_세제곱'] = df_features['기온'] ** 3
    df_features['여름_오후'] = df_features['여름'] * df_features['오후']
    df_features['겨울_저녁'] = df_features['겨울'] * df_features['저녁']
    df_features['주말_오전'] = df_features['주말'] * df_features['오전']
    
    # 결측치 처리 (지연 특성으로 인한)
    df_features = df_features.fillna(method='bfill').fillna(method='ffill')
    
    return df_features


## 모델 학습시간 단축을 위해 최근 연도만 가져옴

In [ ]:
df = df[df["연도"] == 2022].sample(1000)
df.shape

In [ ]:
df.hist(figsize=(15, 15), bins=50);

In [23]:
# 결측치 처리 (예시: 평균값 대체)
df['상대습도'] = df['상대습도'].fillna(df['상대습도'].mean())
df['풍속'] = df['풍속'].fillna(df['풍속'].mean())
df['기온'] = df['기온'].fillna(df['풍속'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
# 데이터 분할
X = df.drop(['전력부하합계', '날짜', '요일'], axis=1)
y = df['전력부하합계']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [26]:
# 수치형 변수만 선택 (범주형/더미는 상황에 따라)
feaure_names = X.columns
X_train_num = X_train[feaure_names]
X_test_num = X_test[feaure_names]

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled = scaler.transform(X_test_num)

## 모델 훈련 및 예측

In [ ]:
# 모델링 및 평가
models = {
    # 'LinearRegression': LinearRegression(),
    # 'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"{name}")
    print(f"R2: {r2_score(y_test, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
    print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred) * 100:.2f}%")
    print("---")

In [28]:
def comprehensive_model_evaluation(X_train_scaled, X_test_scaled, y_train, y_test):
    """
    종합적인 모델 평가 시스템
    """
    # 평가 지표 계산 함수
    def calculate_metrics(y_true, y_pred):
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        mape = mean_absolute_percentage_error(y_true, y_pred) * 100
        return r2, rmse, mae, mape
    
    # 다양한 모델 정의
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge': Ridge(alpha=1.0, random_state=42),
        'Lasso': Lasso(alpha=0.01, random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'Extra Trees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    }
    
    results = {}
    
    print("모델 평가 진행 중...")
    for name, model in models.items():
        # 모델 훈련
        model.fit(X_train_scaled, y_train)
        
        # 예측
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
        
        # 성능 계산
        train_r2, train_rmse, train_mae, train_mape = calculate_metrics(y_train, y_pred_train)
        test_r2, test_rmse, test_mae, test_mape = calculate_metrics(y_test, y_pred_test)
        
        results[name] = {
            'Train_R²': train_r2,
            'Test_R²': test_r2,
            'RMSE': test_rmse,
            'MAE': test_mae,
            'MAPE': test_mape,
            'Overfitting': train_r2 - test_r2,
            'model': model
        }
        
        print(f"{name:<20} Test R²:{test_r2:6.3f} RMSE:{test_rmse:7.2f} MAPE:{test_mape:6.2f}%")
    
    return results


## 시계열 검증

In [ ]:
# 시계열 검증 (TimeSeriesSplit 예시)
tscv = TimeSeriesSplit(n_splits=5)
for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=tscv, scoring='r2')
    print(f"{name} TimeSeries CV R2: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

In [30]:
def hyperparameter_optimization(X_train_scaled, y_train):
    """
    시계열 교차 검증을 활용한 하이퍼파라미터 최적화
    """
    print("하이퍼파라미터 최적화 진행 중...")
    
    # 시계열 교차 검증
    tscv = TimeSeriesSplit(n_splits=3)
    
    # Random Forest 최적화
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [8, 10, 12, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
    grid_search = GridSearchCV(
        rf_base, param_grid, cv=tscv, scoring='r2', n_jobs=-1, verbose=1
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    print(f"최적 파라미터: {grid_search.best_params_}")
    print(f"최적 CV 점수: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_


## 특성 중요도

In [ ]:
# 특성 중요도 (RandomForest)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
importances = pd.Series(rf.feature_importances_, index=feaure_names)
print(importances.sort_values(ascending=False))

In [ ]:
plt.figure(figsize=(10, 6))
importances.sort_values(ascending=True).plot(kind='barh')
plt.title('Feature Importances (RandomForest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [33]:
def create_ensemble_model(X_train_scaled, y_train):
    """
    앙상블 기법을 활용한 모델 성능 향상
    """
    print("앙상블 모델 생성 중...")
    
    # 다양한 모델 조합
    ensemble_models = [
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
        ('ridge', Ridge(alpha=1.0, random_state=42))
    ]
    
    # Voting Regressor 생성
    voting_regressor = VotingRegressor(ensemble_models)
    voting_regressor.fit(X_train_scaled, y_train)
    
    return voting_regressor


In [ ]:
# 고급 특성 공학
df_engineered = advanced_feature_engineering(df)

# 데이터 준비
X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, scaler = prepare_modeling_data(df_engineered)

# 모델 평가
model_results = comprehensive_model_evaluation(X_train_scaled, X_test_scaled, y_train, y_test)

# 하이퍼파라미터 최적화
best_model = hyperparameter_optimization(X_train_scaled, y_train)

# 앙상블 모델
ensemble_model = create_ensemble_model(X_train_scaled, y_train)

# 시계열 교차 검증
tscv = TimeSeriesSplit(n_splits=3)
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=tscv, scoring='r2')

print(f"\n최종 모델 성능:")
print(f"시계열 교차검증 R²: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# 9. 모델 저장 (pickle 사용)
import pickle
with open('best_power_prediction_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("모델 및 스케일러 저장 완료")
